In [ ]:
!pip install pyspark # installing pyspark.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 10.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=9b9cc98b2147520e4e77869c63f07394727363601cc6de283f89d4225bb02ada
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
import sys
from pyspark import SparkContext, SparkConf #installing necessary packages.

In [ ]:
conf = SparkConf()
conf.setMaster('local')
conf.setAppName('CTF of task-1') #setting  and initializing spark context variable.
sc = SparkContext(conf=conf)

In [ ]:
file1="/content/Assignment3_Data.txt"
fileRDD = sc.textFile(file1) # parallelizing raw data into rdd


In [ ]:
#fileRDD.count()

#Task-1

In [ ]:
import re
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
#nltk.data.path.append('/home/schitya/nltk_data') #should specify your nltk downloaded path here.
stop_words = list(stopwords.words('english')) # making list of stop words to filter later
def filtering(x):
  y=x.split('</docid>') 
  z=y[1].split('</text>')
  main=z[0].split('<text>') # whole matter will be in first index of main text.
  #text=main[1].split(' ')
  text_content_body =main[1].split(" ") # splitting the main text based on space.
  
  text = re.sub('\[\[(.*?)\]\]', lambda match: match.group(1), main[1]) #https://stackoverflow.com/questions/2094975/python-re-sub-question #Now we have many [[]] and {{}} this type of words the following functions will extract the word 
  text = re.sub('\{\{(.*?)\}\}', lambda match: match.group(1), text) #https://stackoverflow.com/questions/2094975/python-re-sub-question  # inside these words.
  text_array = text.split(" ")
  filtered_text = ""
  for x1 in text_array:
    if '|' in x1:
      y1 = x1.split('|')
      filtered_text = filtered_text + " ".join(y1)  # eleminating the strings based on '|' if they have any in the word.
    else:
      filtered_text = filtered_text + x1 + " "
  filtered_text = filtered_text.strip() #removing empty spaces at first and at end.
  filtered_text = filtered_text.lower() #converting all the characters to lower 
  
  array = filtered_text.split(' ')  #making the resultant string into List of words.

  text_without_stopwords =""

  for x1 in array:
    if x1 in stop_words:
      continue                           #removing stopwords means if any word is in stopwords list it is not appended as string.
    else:
      text_without_stopwords += x1 + " "

  
  urls = re.compile(r'https?://\S+|www\.\S+') #first using regexs finding websites and likns.
  text_without_urls = urls.sub('', text_without_stopwords) # removing urls by substituing any url with no space.

  text_without_punctuations = re.sub(f"[{re.escape(string.punctuation)}]", " ", text_without_urls) #https://www.geeksforgeeks.org/python-remove-punctuation-from-string/ #Removed punctuations.



  array2 = text_without_punctuations.split(' ') 
  #regex = re.compile('[^a-z]')                              #https://stackoverflow.com/questions/22520932/python-remove-all-non-alphabet-chars-from-string
  useful_text = ""
 
  for x1 in array2:
    useful_text = useful_text + " " + str(re.sub("[^a-zA-Z]+", "", x1)) # after all necessary text processing I am making useful string.
  
  #print(y[0])
  #print(useful_text)
  #print(y[1])

  id_array = y[0].split('<docid>') 
  id = id_array[1]
  return (id, useful_text) # I am returning as tuple where first element is document id and second element is text for further tasks.


  


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
task1RDD = fileRDD.map(lambda x : filtering(x)) #whole articles are filtered as rdd of tuples in which first element is document id and second element is processed text.
#task1RDD = task1RDD.flatMap(lambda x : (x[0],x[1]))

Step-2

In [ ]:
import math
def term_frequency_construction(x):
  content = x[1] # here we get x as tuples in which first element is document id and second element is text content.
  word_frequency = {} # dictionary for calculating the frequency of words.
  content_array= content.split(" ") #splitting based on the empty space for frequency calculation.
  for y in content_array:
    if(len(y)>0):
      if y not in word_frequency:  
        word_frequency[y]=1
      else:                            #first frequency of words are calculated by using dictionary in which if words are not there means assigning it's respective value to be 1 and incrementing the value if key is found
        word_frequency[y] +=1

  sum_of_square_of_euclidean_distance =0 
  for y in word_frequency:
    sum_of_square_of_euclidean_distance += word_frequency[y]*word_frequency[y]  # for every value in dictionary calculating sum of square of euclidean distances.
  euclidean_distance = math.sqrt(sum_of_square_of_euclidean_distance) # sqrt of all sum of squares is done for exact euclidean distance.

  for y in word_frequency:
    count = word_frequency[y] 
    freq = 1+ math.log(count,10)
    normalized_freq = freq/euclidean_distance #every value in dictionary is normalized by calculated euclidean distances. 
    word_frequency[y]= normalized_freq
 

  return (x[0], word_frequency) #returning document id in first element of tuple and dictionary of (1+log(freq))/euclidean_distance values and words as keys in second element 



In [ ]:
step2RDD = task1RDD.map(lambda x : term_frequency_construction(x))#constructing term frequecy matrix and we get rdd of tuples of first element as doc-id and second element as dicitonary of words with normalization of log of frequency values.
step2RDD = step2RDD.flatMap(lambda x : [(word, x[0], x[1][word]) for word in x[1]]) #Now we need every word in dictionary of every element of tuple as key so for that I used for loop in lambda function and for every word I made tuple of word in dictionary of every element of rdd, document-id as second element and value as third element.
step2RDD = step2RDD.map(lambda x: (x[0],(x[1],x[2]))) #After flatting I made every a tuple like (word, (doc-id,freq))
step2RDD = step2RDD.map(lambda x: (x[0], [x[1]])) #After above transformation I will have rdd of many elements like of the format (word,(doc-id,freq)) that means I can have same word as key with different tuple as value so I am making list for values to add for same key.
step2RDD = step2RDD.reduceByKey(lambda a, b: a + b) ##For same key I will add (doc-id,freq) tuples to the list that I have created above.

#task2RDD = task2RDD.map(lambda x : (x[0], [(x[0], v) for v in x[1:][0]]))

In [ ]:
#task2RDD.take(1)

In [ ]:
def conversion(x): #we get every line as tuple of the form (word,[(doc-1,freq),(doc-2,freq).......])
  first_part = str(x[0]) + "@"
  remaining_part = "" 
  for y in x[1]:
    if len(y)==2:
      t=""
      t = "doc" + "-" + y[0] + "#" + str(y[1]) + "+"  # using for loop to convert in required format.
      remaining_part = remaining_part + t
    main = first_part + remaining_part            

  main = main[:-1] #removing last "+" that will be added in the for loop at end.
  return main




In [ ]:
step2RDD = step2RDD.map(lambda x : conversion(x)) # every rdd element is converted to required string format.

In [ ]:
#step2RDD.take(1)

In [ ]:
import os
folder_path = "/content/CTF_index"
step2RDD.saveAsTextFile(folder_path) # rdd is writen to specified folder for using for TF_Query.